In [224]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import re
import matplotlib.pyplot  as plt
%matplotlib inline
import requests

In [443]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

In [225]:
#Directory
path = os.getcwd() 
data_root = path + '\\DATA\\'
data_root

'C:\\Users\\Kaipable\\Desktop\\GitHub\\voter-protection-corps\\pollworkers_src\\DATA\\'

##  Common Congressional Election Survey Data

In [226]:
# Read in CCES data
cces = pd.read_csv(data_root + 'CCES16_Common_OUTPUT_Feb2018_VV.tab', sep='\t')

#Explore 
print(cces.shape)
print(cces.columns)
cces.head(5)

C:\Users\Kaipable\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (46,47,48,49,50,51,52,53,54,55,370,371,373,374,375,376,377,378,379,380,381,382,383,385,386,388,389,390,391,393,394,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,419,420,422,423,424,425,426,427,430,432,436,439,441,443,445,446,447,448,449,450,451,452,453,454,455,456,458,460,462,486,487,493,494,513,515,520,522,526,527,528,533,534,535,541,542,543,544,545,559,560) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['V101', 'commonweight_vv', 'commonweight_vv_post',
       'commonweight_vv_lgbt', 'commonweight', 'commonweight_post', 'tookpost',
       'lookupzip', 'inputstate', 'cdid113',
       ...
       'CL_E2016PPEP', 'CL_E2016PPVM', 'CL_E2016PEP', 'CL_E2016PVM',
       'starttime_pre', 'endtime_pre', 'starttime_post', 'endtime_post', 'dma',
       'dmaname'],
      dtype='object', length=563)
64600


,V101,commonweight_vv,commonweight_vv_post,commonweight_vv_lgbt,commonweight,commonweight_post,tookpost,lookupzip,inputstate,cdid113,...,CL_E2016PPEP,CL_E2016PPVM,CL_E2016PEP,CL_E2016PVM,starttime_pre,endtime_pre,starttime_post,endtime_post,dma,dmaname
0,222168628,1.344214,1.042267,1.195346,0.944967,0.730450,1,3442,33,2,...,NaN,NaN,NaN,NaN,2016-10-20 05:05:44.887,2016-10-20 05:36:38.850,2016-11-16 06:24:32.895,2016-11-16 06:40:18.810,NaN,NaN
1,273691199,1.182352,1.280103,1.280565,0.767256,0.892838,1,70118,22,2,...,NaN,NaN,NaN,NaN,2016-10-23 14:54:58.548,2016-10-24 14:20:41.711,2016-11-30 14:02:55.143,2016-11-30 14:09:26.931,622.0,NEW ORLEANS
2,284214415,0.217396,NaN,0.248930,0.090604,0.000000,0,63851,29,8,...,NaN,NaN,NaN,NaN,2016-10-29 06:54:12.797,2016-10-29 07:24:44.759,NaN,NaN,632.0,PADUCAH-CAPE GIRARD-HARSBG
3,287557695,0.532320,NaN,0.928750,0.142583,0.000000,0,36703,1,7,...,NaN,NaN,NaN,NaN,2016-10-27 11:06:21.322,2016-10-27 11:26:00.051,NaN,NaN,698.0,MONTGOMERY-SELMA
4,290387662,1.261106,1.092519,1.161894,1.206498,1.019007,1,80526,8,2,...,NaN,NaN,NaN,NaN,2016-10-10 10:56:31.061,2016-10-10 11:14:12.215,2016-11-18 09:09:28.044,2016-11-18 09:21:05.905,751.0,DENVER


In [227]:
# Common Field
cces.countyfips[0:5]

0    33011.0
1    22071.0
2    29155.0
3     1047.0
4     8069.0
Name: countyfips, dtype: float64

In [228]:
#Clean Up County Code
cces['clean_county'] = cces['countyfips'].apply(lambda x: str(x).strip('.0')[-3:]   if len(str(x)) > 6 else '0'+ str(x).strip('.0')[0:2])
print(cces['clean_county'][0:5])

# Clean Up State Codes
cces['clean_state'] = cces['inputstate'].apply(lambda x: str(x) if len(str(x)) > 1 else '0'+ str(x))
#print(cces['clean_state'][0:5])

# Altogether;  Cretae FIPS Column
cces['FIPSCode'] = cces['clean_state'] + cces['clean_county']

#Match Type of DF
#cces['FIPSCode']= pd.to_numeric(cces['FIPSCode'], errors='coerce').fillna(0).astype(np.int64)
print(cces['FIPSCode'][0:5])

0    011
1    071
2    155
3    010
4    080
Name: clean_county, dtype: object
0    33011
1    22071
2    29155
3    01010
4    08080
Name: FIPSCode, dtype: object


In [229]:
#Investigate Voter Wait Time Column
percent_null = cces['CC16_404'].isna().sum() / len(cces.index)

print('Of the',len(cces.index), 'rows in the CCES dataset,', np.round(percent_null*100,2), '% of wait time entries are null.')
print('CCES has data from',cces['FIPSCode'].nunique(), 'counties')

#Categorical Data Codes
vote_dict = {1:0, 2: 'Less than 10', 3: '10-30 minutes', 4: '31 minutes - 1 hour', 5: 'more than one hour', 6: None}

Of the 64600 rows in the CCES dataset, 46.82 % of wait time entries are null.
CCES has data from 2408 counties


In [445]:
#DF for Just Vote Times to Merge
vote_times = cces[['V101','inputstate','FIPSCode','CC16_404']]
vote_times['CC16_404'] = vote_times['CC16_404'].replace(6,np.NaN) # 6 equates to NaN
vote_times = vote_times.dropna()

# Interpretable variable names. 
vote_times.columns = ['caseid','state','FIPSCode', 'wait_time'] 

#Did it Work?
vote_times.describe()

vote_times['wait_time'].dtype
print('We have vote times for',vote_times['FIPSCode'].nunique(), 'counties')
vote_times.head()

We have vote times for 2182 counties


C:\Users\Kaipable\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,caseid,state,FIPSCode,wait_time
0,222168628,33,33011,2.0
1,273691199,22,22071,1.0
4,290387662,8,08080,1.0
6,292860642,48,48231,2.0
8,295717127,13,13213,2.0


In [233]:
# Average Times by County
vote_by_fips = vote_times.groupby('FIPSCode', 
                as_index=False)['wait_time'].agg({'wait_time':'mean',
                  'wait_count': 'count'})
vote_by_fips.head()

,FIPSCode,wait_time,wait_count
0,01010,2.250720,347
1,01011,1.953846,130
2,02020,1.840909,44
3,02021,1.695652,23
4,02022,1.000000,2


# EAVS Dataset

#### Poll Worker Age
* D8a. TOTAL number of poll workers  
* D8b. Under 18 years old  
* D8c. 18 to 24  
* D8d. 26 to 40  
* D8e. 41 to 60  
* D8f. 61 to 70  
* D8g. 71 years old and over  

#### Section A: Voter Registration  
General info about registration numbers, forms processed, mode, and decision  
* A1a Total  Registered Voters
* Number of Active (A1b) and Inactive Voters (A1c)  
** Percent inactive (A1c/A1a) could communicate folks whose registration will take longer -- longer wait times > more resources
* % Same Day Registration
* Total # of same day registration (A2a)
* all registered voters (A1a)
* all new registrations (A3b)
* total processed registrations (A3a) in time frame

* Perhaps as percentage of registrations (A3b)  


#### Section D: Total Votes Cast an In-Person Voting  
Includes number of in-person voters, # precincts, # of polling places, # of poll workers, # in-person absentee  
* Total number of votes cast (D1a)  
* Total number voting in-person, physical voters (D2)
* In-person on election day (D2a)
* Early vote in-person (D2b)
* of Precincts (D3)
* Physical Polling Places (D4a)
Early vote (D5a) 
* Non-election office polling places (D5c)    

#### POLL WORKERS  
* Total # Poll workers (D8a)  
For election day (D6)  
For early voting (D7)  
* Ease of Recruiting Poll Workers (D9)  

#### PARTICIPATION  
* F1a. TOTAL number of voters participating
* F1b. Voted at a physical polling place on Election Day  


#### Summary:  New Vars to Add
* In Person Election Day Votes / Total Cotes Cast = D2 / D1 
    - In Person Election Day VOTERS / Total Voters = F1b/F1a
* Total Number of Poll Workers = D8a
* PENDING: Technology



In [365]:
#df = pd.read_excel('../DATA/EAVS_2018_for_Public_Release.xlsx')
df = pd.read_excel(data_root +'EAVS_2018_for_Public_Release_Updates.xlsx', dtype=str)

#eXPLORE
print(df.shape)
df.describe()
df.head(30)

6460


In [449]:
#How many municipalities per each state? 
num_muni = df.groupby(['State_Abbr']).count()
num_muni.head()

,FIPSCode_x,Jurisdiction_Name,State_Full,A1a,A1b,A1c,A1Comments,A2a,A2Comments,A3a,A3b,A3c,A3d,A3e,A3f,A3g,A3h_Other,A3h,A3i_Other,A3i,A3j_Other,A3j,A3Comments,A4a,A4b,A4c,A4d,A4e,A4f,A4g,A4h,A4i,A4j_Other,A4j,A4k_Other,A4k,A4l_Other,A4l,A5a,A5b,A5c,A5d,A5e,A5f,A5g,A5h,A5i,A5j_Other,A5j,A5k_Other,A5k,A5l_Other,A5l,A6a,A6b,A6c,A6d,A6e,A6f,A6g,A6h,A6i,A6j_Other,A6j,A6k_Other,A6k,A6l_Other,A6l,A7a,A7b,A7c,A7d,A7e,A7f,A7g,A7h,A7i,A7j_Other,A7j,A7k_Other,A7k,A7l_Other,A7l,A4_A7Comments,A8a,A8b,A8c,A8d,A8e,A8f_Other,A8f,A8g_Other,A8g,A8h_Other,A8h,A8Comments,A9a,A9b,A9c,A9d,A9e,A9f,A9g,A9h_Other,A9h,A9i_Other,A9i,A9j_Other,A9j,A9Comments,B1a,B1b,B1c,B1Comments,B2a,B2b,B2c,B2Comments,B3a,B3b,B3c,B3Comments,B4a,B4Comments,B5a,B5b,B5c,B6a,B6b,B6c,B7a,B7b,B7c,B8a,B8b,B8c,B5_B8Comments,B9a,B9b,B9c,B10a,B10b,B10c,B11a,B11b,B11c,B12a,B12b,B12c,B9_B12Comments,B13a,B13b,B13c,B13d,B13Comments,B14a,B14b,B14c,B15a,B15b,B15c,B16a,B16b,B16c,B17a,B17b,B17c,B14_B17Comments,B18a,B18b,B18c,B19a,B19b,B19c,B20a,B20b,B20c,B21a,B21b,B21c,B22a,B22b,B22c,B22_Other,B18_B22Comments,B23a,B23b,B23c,B24a,B24b,B24c,B25a,B25b,B25c,B26a,B26b,B26c,B23_B26Comments,C1a,C1b,C1c,C1d,C1e,C1f,C1g_Other,C1g,C1h_Other,C1h,C1i_Other,C1i,C1Comments,C2a,C2Comments,C3a,C3Comments,C4a,C4b,C4c,C4d,C4e,C4f,C4g,C4h,C4i,C4j,C4k,C4l,C4m,C4n,C4o,C4p_Other,C4p,C4q_Other,C4q,C4r_Other,C4r,C4Comments,D1a,D1Comments,D2a,D2b,D2Comments,D3a,D3Comments,D4a,D4b,D4c,D5a,D5b,D5c,D4_D5Comments,D6,D7,D6_D7Comments,D8a,D8b,D8c,D8d,D8e,D8f,D8g,D8Comments,D9,D9Comments,E1a,E1b,E1c,E1d,E1e_Other,E1e,E1Comments,E2a,E2b,E2c,E2d,E2e,E2f,E2g,E2h,E2i,E2j,E2k_Other,E2k,E2l_Other,E2l,E2m_Other,E2m,E2Comments,F1a,F1b,F1c,F1d,F1e,F1f,F1g,F1h_Other,F1h,F1Comments,F2,F2_Other,F2Comments,F3a,F3b,F3c,F3d_Other,F3d,F4a,F4b,F4c,F4d_Other,F4d,F3_F4Comments,F5a,F5b_1,F5b_1other,F5c_1,F5b_2,F5b_2other,F5c_2,F5b_3,F5b_3other,F5c_3,F5d_1,F5d_2,F5d_3,F5d_4,F6a,F6b_1,F6b_1other,F6c_1,F6b_2,F6b_2other,F6c_2,F6b_3,F6b_3other,F6c_3,F6d_1,F6d_2,F6d_3,F6d_4,F7a,F7b_1,F7b_1other,F7c_1,F7b_2,F7b_2other,F7c_2,F7b_3,F7b_3other,F7c_3,F7d_1,F7d_2,F7d_3,F7d_4,F7d_5,F8a,F8b_1,F8b_1other,F8c_1,F8b_2,F8b_2other,F8c_2,F8b_3,F8b_3other,F8c_3,F8d_1,F8d_2,F8d_3,F8d_4,F8d_5,F9a,F9b_1other,F9c_1,F9b_2other,F9c_2,F9b_3other,F9c_3,F9d_1,F9d_2,F9d_3,F9d_4,F9d_5,F10a,F10b_1other,F10c_1,F10b_2other,F10c_2,F10b_3other,F10c_3,F10d_1,F10d_2,F10d_4,F11a,F11d_1,F11d_2,F11d_3,F11d_4,F11d_5,F5_F11Comments,F12a,F12b,F12c,F12d,F12e,F12Comments,F13,short_fips,eligible_voters,num_inactive,num_same_day_pollworkers,num_machines,in_person_voters,num_pollworkers,num_same_day_locations,num_pw_old,voters_removed,num_precincts,total_mail_ballots,num_in_person,percent_in_person,percent_inactive,percent_early_vote,percent_rejected_ballot,difficulty,voter_per_location,voter_per_machine,voter_per_pw,total_resources_per_voter,frac_pw_old,FIPSCode_y,wta_counts,FIPSCode,wait_time,wait_count
State_Abbr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AK,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,0,1,1,1,1,1

In [367]:
# Clean Up
df = df.replace('Does not apply', None)
df = df.replace('Data not available', np.nan)
df = df.replace('NaN', None)
df = df.replace('Data not available', None)

In [368]:
# EAVS fields of Interest
eavs_vars = [ 'A1a', 'A1c', 'D1a','D2a','D2b', 'D4a', 'D4b', 'D6','D8f', 'D8g','D8a','F1b', 'F1a', 'F5c_1', 'F6c_1','F7c_1', 'F8c_1', 'F9c_1', 'F10c_1','F5c_2', 'F6c_2','F7c_2', 'F8c_2', 'F9c_2', 'F10c_2','F5c_3', 'F6c_3','F7c_3', 'F8c_3', 'F9c_3', 'F10c_3', 'D3a', 'C1a', 'C4a']

#Type Conversion
df[eavs_vars] = df[eavs_vars].astype('float64')

# All vars containing # of machines contain C_
machine_vars = [k for k in eavs_vars if 'c_' in k ]

###  Wisconsin Procesing
- WI has "refused" for all difficulty columns

In [369]:
#Prepare to aggregate numeric columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']


# Isolate Wisconsin town-level data  and remove from df for processing/aggregating
wi_df = df[df['State_Abbr']=='WI']
df =  df[df['State_Abbr']!='WI']

#Create Common Column; EAVS has long and short form FIPS Codes
df['short_fips'] =df.FIPSCode.apply(lambda x: str(x)[0:5])
wi_df['Jurisdiction Name'] = wi_df.Jurisdiction_Name.str.split("- ").str[1]
wi_df['State_Code'] = '55'
wi_df['Jurisdiction_Name'] = wi_df['Jurisdiction Name']


#Get True FIPS from external  source and merge
wi_names = pd.read_csv(data_root +'wi_counties.csv')
wi_names['Full_County_Name'] = wi_names['Full_County_Name'].astype(str)
wi = wi_df.merge(wi_names, how="left", left_on='Jurisdiction_Name', right_on='Full_County_Name')

#Clean and structure
wi = wi.dropna(subset=['short_fips'])
wi['short_fips'] = wi['short_fips'].astype(int)
wi['short_fips'] = wi['short_fips'].astype(str)
wi = wi.set_index(['short_fips', 'Jurisdiction_Name', 'State_Full', 'State_Abbr'])
wi = wi.select_dtypes(include=numerics)
wi.head()

A1a     A1c    C1a  \
short_fips Jurisdiction_Name State_Full State_Abbr                          
55083      OCONTO COUNTY     WISCONSIN  WI          1218.0  5636.0  118.0   
55067      LANGLADE COUNTY   WISCONSIN  WI           325.0  5636.0   15.0   
55001      ADAMS COUNTY      WISCONSIN  WI           905.0  5636.0   42.0   
                                        WI           740.0  5636.0   33.0   
55045      GREEN COUNTY      WISCONSIN  WI           337.0  5636.0   15.0   

                                                    C4a    D1a    D2a   D2b  \
short_fips Jurisdiction_Name State_Full State_Abbr                            
55083      OCONTO COUNTY     WISCONSIN  WI          0.0  919.0  795.0   0.0   
55067      LANGLADE COUNTY   WISCONSIN  WI          0.0  266.0  247.0   4.0   
55001      ADAMS COUNTY      WISCONSIN  WI          0.0  675.0  574.0  68.0   
                                        WI          3.0  609.0  581.0   0.0   
55045      GREEN COUNTY      WISCONSIN  WI          0.0  279.0  267.0   6.0   

                                                    D3a  D4a  D4b   ...    \
short_fips Jurisdiction_Name State_Full State_Abbr                  ...     
55083      OCONTO COUNTY     WISCONSIN  WI          1.0  1.0  1.0   ...     
55067      LANGLADE COUNTY   WISCONSIN  WI          1.0  1.0  1.0   ...     
55001      ADAMS COUNTY      WISCONSIN  WI          1.0  1.0  1.0   ...     
                                        WI          1.0  1.0  1.0   ...     
55045      GREEN COUNTY      WISCONSIN  WI          1.0  1.0  1.0   ...     

                                                    F7c_3  F8c_1  F8c_2  \
short_fips Jurisdiction_Name State_Full State_Abbr                        
55083      OCONTO COUNTY     WISCONSIN  WI            NaN    NaN    NaN   
55067      LANGLADE COUNTY   WISCONSIN  WI            NaN    NaN    NaN   
55001      ADAMS COUNTY      WISCONSIN  WI            NaN    NaN    NaN   
                                        WI            NaN    NaN    NaN   
55045      GREEN COUNTY      WISCONSIN  WI            NaN    NaN    NaN   

                                                    F8c_3  F9c_1  F9c_2  \
short_fips Jurisdiction_Name State_Full State_Abbr                        
55083      OCONTO COUNTY     WISCONSIN  WI            NaN    NaN    NaN   
55067      LANGLADE COUNTY   WISCONSIN  WI            NaN    NaN    NaN   
55001      ADAMS COUNTY      WISCONSIN  WI            NaN    NaN    NaN   
                                        WI            NaN    NaN    NaN   
55045      GREEN COUNTY      WISCONSIN  WI            NaN    NaN    NaN   

                                                    F9c_3  F10c_1  F10c_2  \
short_fips Jurisdiction_Name State_Full State_Abbr                          
55083      OCONTO COUNTY     WISCONSIN  WI            NaN     NaN     NaN   
55067      LANGLADE COUNTY   WISCONSIN  WI            NaN     NaN     NaN   
55001      ADAMS COUNTY      WISCONSIN  WI            NaN     NaN     NaN   
                                        WI            NaN     NaN     NaN   
55045      GREEN COUNTY      WISCONSIN  WI            NaN     NaN     NaN   

                                                    F10c_3  
short_fips Jurisdiction_Name State_Full State_Abbr          
55083      OCONTO COUNTY     WISCONSIN  WI             NaN  
55067      LANGLADE COUNTY   WISCONSIN  WI             NaN  
55001      ADAMS COUNTY      WISCONSIN  WI             NaN  
                                        WI             NaN  
55045      GREEN COUNTY      WISCONSIN  WI             NaN  

[5 rows x 34 columns]

In [450]:
#Prepare to Aggregate
sum_list = [ name for name in wi.columns if name != 'D9']
agg_dict = { var:'sum' for var in sum_list}
wi_agg = wi.reset_index().groupby(['short_fips', 'Jurisdiction_Name', 'State_Full', 'State_Abbr']).agg(agg_dict).reset_index()

#Check Output
wi_agg.head()

,short_fips,Jurisdiction_Name,State_Full,State_Abbr,A1a,A1c,C1a,C4a,D1a,D2a,D2b,D3a,D4a,D4b,D6,D8a,D8f,D8g,F1a,F1b,F5c_1,F5c_2,F5c_3,F6c_1,F6c_2,F6c_3,F7c_1,F7c_2,F7c_3,F8c_1,F8c_2,F8c_3,F9c_1,F9c_2,F9c_3,F10c_1,F10c_2,F10c_3
0,55001,ADAMS COUNTY,WISCONSIN,WI,12029.0,107084.0,570.0,6.0,9529.0,8344.0,656.0,19.0,19.0,19.0,0.0,0.0,0.0,0.0,9529.0,8344.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,55003,ASHLAND COUNTY,WISCONSIN,WI,4744.0,84540.0,169.0,4.0,3547.0,3245.0,155.0,15.0,15.0,15.0,0.0,0.0,0.0,0.0,3547.0,3245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,55005,BARRON COUNTY,WISCONSIN,WI,25198.0,191624.0,954.0,21.0,18398.0,15838.0,1706.0,34.0,34.0,34.0,0.0,0.0,0.0,0.0,18398.0,15838.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,55007,BAYFIELD COUNTY,WISCONSIN,WI,11268.0,157808.0,638.0,12.0,8861.0,7757.0,518.0,28.0,28.0,28.0,0.0,0.0,0.0,0.0,8861.0,7757.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,55009,BROWN COUNTY,WISCONSIN,WI,134434.0,118356.0,7524.0,92.0,104158.0,78842.0,18568.0,87.0,62.0,62.0,0.0,0.0,0.0,0.0,104158.0,78842.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [371]:
#Add cleaned Wisconsin data to main dataframe
df = df.append(wi_agg, sort=False)

In [452]:
#Check
df[df['State_Abbr']=='WI'].short_fips[0:5]

4610    55001
4611    55003
4612    55005
4613    55007
4614    55009
Name: short_fips, dtype: object

In [373]:
# Better Labels for EAVS fields
df['eligible_voters'] = df['A1a']
df['num_inactive'] = df['A1c']
df['num_same_day_pollworkers'] = df['D6']
df['num_machines'] = df[machine_vars].sum(axis=1, skipna=True)
df['in_person_voters'] = df['D2a'] #
df['num_pollworkers'] = np.where(pd.notna(df['num_same_day_pollworkers']), df['num_same_day_pollworkers'], df['D8a'])
df['num_same_day_locations'] = df['D4a']
df['num_pw_old'] = df['D8f'] + df['D8g']
df['voters_removed'] = df['A9a']
df['num_precincts'] = df['D3a']
df['total_mail_ballots'] =  df['C3a']
df['num_in_person'] = df['F1b'] 


# Engineered Features
#Historical Characteristics
df['percent_in_person'] = df['F1b'] / df['F1a']
df['percent_inactive'] = df['num_inactive'] / df['eligible_voters']

df['percent_early_vote'] = df['D2b'] / df['D1a']
df['percent_rejected_ballot'] = df['C4a'] / df['C1a']

# Recode Difficulty Survey
difficulty = {
    'Very difficult':5,
    'Somewhat difficult':4,
    'Neither difficult nor easy':3,
    'Somewhat easy':2,
    'Very easy':1,
    'Not enough information to answer':None  
}
df['difficulty'] = df['D9'].map(difficulty)
#df.difficulty.unique()

#Resources
df['voter_per_location'] = df['eligible_voters'] / df['num_same_day_locations']
df['voter_per_machine'] =  df['eligible_voters']/ df['num_machines']
df['voter_per_pw'] = np.where(pd.notna(df['num_same_day_pollworkers']), df['eligible_voters'] / 
                    df['num_same_day_pollworkers'], df['eligible_voters']/ df['num_pollworkers'] )

#Blended
df['total_resources_per_voter'] = df['eligible_voters'] / (df['num_machines'] + df['num_pollworkers'])

#Demographic
df['num_pw_old'] = df['D8f'] + df['D8g']
df['frac_pw_old'] = df['num_pw_old'] / df['D8a']

In [374]:
df.total_resources_per_voter[0:5]

0    242.500000
1    242.007800
2    139.701613
3    158.266667
4    224.220339
Name: total_resources_per_voter, dtype: float64

### Merge all data across sources

In [375]:
# Cast FIPS to Integer
df.short_fips = df.short_fips.astype(str)
vote_by_fips.FIPSCode = vote_by_fips.FIPSCode.astype(str)
wta_resources.FIPSCode = wta_resources.FIPSCode.astype(str)

In [377]:
# Merge with Vote Time
df = df.merge(vote_by_fips, how="left", left_on='short_fips', right_on='FIPSCode')
df.head(50)

,FIPSCode_x,Jurisdiction_Name,State_Full,State_Abbr,A1a,A1b,A1c,A1Comments,A2a,A2Comments,...,voter_per_location,voter_per_machine,voter_per_pw,total_resources_per_voter,frac_pw_old,FIPSCode_y,wta_counts,FIPSCode,wait_time,wait_count
0,0100100000,AUTAUGA COUNTY,ALABAMA,AL,39770.0,36391,3379.0,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,2093.157895,864.565217,337.033898,242.500000,0.813559,NaN,NaN,NaN,NaN,NaN
1,0100300000,BALDWIN COUNTY,ALABAMA,AL,155127.0,141301,13826.0,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,3165.857143,1251.024194,300.052224,242.007800,0.820116,NaN,NaN,NaN,NaN,NaN
2,0100500000,BARBOUR COUNTY,ALABAMA,AL,17323.0,15871,1452.0,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,184.287234,524.939394,190.362637,139.701613,0.791209,NaN,NaN,NaN,NaN,NaN
3,0100700000,BIBB COUNTY,ALABAMA,AL,14244.0,13245,999.0,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,1780.500000,749.684211,200.619718,158.266667,0.732394,NaN,NaN,NaN,NaN,NaN
4,0100900000,BLOUNT COUNTY,ALABAMA,AL,39687.0,37352,2335.0,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,1526.423077,763.211538,317.496000,224.220339,0.832000,NaN,NaN,NaN,NaN,NaN
5,0101100000,BULLOCK COUNTY,ALABAMA,AL,7472.0,6772,700.0,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,111.522388,149.440000,73.254902,49.157895,0.558824,NaN,NaN,01011,1.953846,130.0
6,0101300000,BUTLER COUNTY,ALABAMA,AL,14079.0,13161,918.0,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,639.954545,281.580000,93.860000,70.395000,0.787500,NaN,NaN,NaN,NaN,NaN
7,0101500000,CALHOUN COUNTY,ALABAMA,AL,76168.0,69851,6317.0,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,1655.826087,785.237113,238.025000,182.657074,0.887500,NaN,NaN,NaN,NaN,NaN
8,0101700000,CHAMBERS COUNTY,ALABAMA,AL,24367.0,22474,1893.0,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,1218.350000,435.125000,487.340000,229.877358,0.240000,NaN,NaN,NaN,NaN,NaN
9,0101900000,CHEROKEE COUNTY,ALABAMA,AL,17893.0,16808,1085.0,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,777.956522,331.351852,144.298387,100.522472,0.750000,NaN,NaN,NaN,NaN,NaN


In [451]:
df.isnull().sum().head()

FIPSCode_x           71
Jurisdiction_Name     0
State_Full            0
State_Abbr            0
A1a                  11
dtype: int64

## ACS Data

In [384]:
acs = pd.read_csv(data_root+ 'all_states_census.csv', dtype='str')

#Explore
print(acs.columns)
print(len(acs))
acs.head()

Index(['Unnamed: 0', 'index', 'DP05_0033E', 'DP05_0038E', 'DP05_0072E',
       'DP03_0063E', 'DP02_0001E', 'DP03_0062E', 'DP05_0018E', 'DP05_0014E',
       'DP05_0015E', 'DP05_0016E', 'DP05_0017E', 'fips_code', 'state_code',
       'Name', 'FIPS State Numeric Code', 'Official USPS Code'],
      dtype='object')
3220


,Unnamed: 0,index,DP05_0033E,DP05_0038E,DP05_0072E,DP03_0063E,DP02_0001E,DP03_0062E,DP05_0018E,DP05_0014E,DP05_0015E,DP05_0016E,DP05_0017E,fips_code,state_code,Name,FIPS State Numeric Code,Official USPS Code
0,0,"Washington County, Mississippi: Summary level:...",47086,33982,579,49433,18299.0,30834,36.9,3291,4282,1961,710,28151,28,Mississippi,28,MS
1,1,"Perry County, Mississippi: Summary level: 050,...",12028,2368,150,55077,4563.0,39007,40.9,897,1247,589,236,28111,28,Mississippi,28,MS
2,2,"Choctaw County, Mississippi: Summary level: 05...",8321,2596,14,49162,3164.0,37203,44.0,644,979,547,214,28019,28,Mississippi,28,MS
3,3,"Itawamba County, Mississippi: Summary level: 0...",23480,1681,320,53717,8706.0,40510,40.0,1435,2250,1181,574,28057,28,Mississippi,28,MS
4,4,"Carroll County, Mississippi: Summary level: 05...",10129,3503,2,57919,3658.0,43060,47.0,623,1353,726,170,28015,28,Mississippi,28,MS


### Understand ACS and  EAVS Incompatibility

In [463]:
eavs_muni.head()

census_counties.head()

df[df['State_Abbr']=='CA']

,FIPSCode_x,Jurisdiction_Name,State_Full,State_Abbr,A1a,A1b,A1c,A1Comments,A2a,A2Comments,A3a,A3b,A3c,A3d,A3e,A3f,A3g,A3h_Other,A3h,A3i_Other,A3i,A3j_Other,A3j,A3Comments,A4a,A4b,A4c,A4d,A4e,A4f,A4g,A4h,A4i,A4j_Other,A4j,A4k_Other,A4k,A4l_Other,A4l,A5a,A5b,A5c,A5d,A5e,A5f,A5g,A5h,A5i,A5j_Other,A5j,A5k_Other,A5k,A5l_Other,A5l,A6a,A6b,A6c,A6d,A6e,A6f,A6g,A6h,A6i,A6j_Other,A6j,A6k_Other,A6k,A6l_Other,A6l,A7a,A7b,A7c,A7d,A7e,A7f,A7g,A7h,A7i,A7j_Other,A7j,A7k_Other,A7k,A7l_Other,A7l,A4_A7Comments,A8a,A8b,A8c,A8d,A8e,A8f_Other,A8f,A8g_Other,A8g,A8h_Other,A8h,A8Comments,A9a,A9b,A9c,A9d,A9e,A9f,A9g,A9h_Other,A9h,A9i_Other,A9i,A9j_Other,A9j,A9Comments,B1a,B1b,B1c,B1Comments,B2a,B2b,B2c,B2Comments,B3a,B3b,B3c,B3Comments,B4a,B4Comments,B5a,B5b,B5c,B6a,B6b,B6c,B7a,B7b,B7c,B8a,B8b,B8c,B5_B8Comments,B9a,B9b,B9c,B10a,B10b,B10c,B11a,B11b,B11c,B12a,B12b,B12c,B9_B12Comments,B13a,B13b,B13c,B13d,B13Comments,B14a,B14b,B14c,B15a,B15b,B15c,B16a,B16b,B16c,B17a,B17b,B17c,B14_B17Comments,B18a,B18b,B18c,B19a,B19b,B19c,B20a,B20b,B20c,B21a,B21b,B21c,B22a,B22b,B22c,B22_Other,B18_B22Comments,B23a,B23b,B23c,B24a,B24b,B24c,B25a,B25b,B25c,B26a,B26b,B26c,B23_B26Comments,C1a,C1b,C1c,C1d,C1e,C1f,C1g_Other,C1g,C1h_Other,C1h,C1i_Other,C1i,C1Comments,C2a,C2Comments,C3a,C3Comments,C4a,C4b,C4c,C4d,C4e,C4f,C4g,C4h,C4i,C4j,C4k,C4l,C4m,C4n,C4o,C4p_Other,C4p,C4q_Other,C4q,C4r_Other,C4r,C4Comments,D1a,D1Comments,D2a,D2b,D2Comments,D3a,D3Comments,D4a,D4b,D4c,D5a,D5b,D5c,D4_D5Comments,D6,D7,D6_D7Comments,D8a,D8b,D8c,D8d,D8e,D8f,D8g,D8Comments,D9,D9Comments,E1a,E1b,E1c,E1d,E1e_Other,E1e,E1Comments,E2a,E2b,E2c,E2d,E2e,E2f,E2g,E2h,E2i,E2j,E2k_Other,E2k,E2l_Other,E2l,E2m_Other,E2m,E2Comments,F1a,F1b,F1c,F1d,F1e,F1f,F1g,F1h_Other,F1h,F1Comments,F2,F2_Other,F2Comments,F3a,F3b,F3c,F3d_Other,F3d,F4a,F4b,F4c,F4d_Other,F4d,F3_F4Comments,F5a,F5b_1,F5b_1other,F5c_1,F5b_2,F5b_2other,F5c_2,F5b_3,F5b_3other,F5c_3,F5d_1,F5d_2,F5d_3,F5d_4,F6a,F6b_1,F6b_1other,F6c_1,F6b_2,F6b_2other,F6c_2,F6b_3,F6b_3other,F6c_3,F6d_1,F6d_2,F6d_3,F6d_4,F7a,F7b_1,F7b_1other,F7c_1,F7b_2,F7b_2other,F7c_2,F7b_3,F7b_3other,F7c_3,F7d_1,F7d_2,F7d_3,F7d_4,F7d_5,F8a,F8b_1,F8b_1other,F8c_1,F8b_2,F8b_2other,F8c_2,F8b_3,F8b_3other,F8c_3,F8d_1,F8d_2,F8d_3,F8d_4,F8d_5,F9a,F9b_1other,F9c_1,F9b_2other,F9c_2,F9b_3other,F9c_3,F9d_1,F9d_2,F9d_3,F9d_4,F9d_5,F10a,F10b_1other,F10c_1,F10b_2other,F10c_2,F10b_3other,F10c_3,F10d_1,F10d_2,F10d_4,F11a,F11d_1,F11d_2,F11d_3,F11d_4,F11d_5,F5_F11Comments,F12a,F12b,F12c,F12d,F12e,F12Comments,F13,short_fips,eligible_voters,num_inactive,num_same_day_pollworkers,num_machines,in_person_voters,num_pollworkers,num_same_day_locations,num_pw_old,voters_removed,num_precincts,total_mail_ballots,num_in_person,percent_in_person,percent_inactive,percent_early_vote,percent_rejected_ballot,difficulty,voter_per_location,voter_per_machine,voter_per_pw,total_resources_per_voter,frac_pw_old,FIPSCode_y,wta_counts,FIPSCode,wait_time,wait_count
159,0600100000,ALAMEDA COUNTY,CALIFORNIA,CA,1003541.0,881491,122050.0,nan,547,nan,853286,75962,6275,356501,6635,76608,63952,DECEASED/UNDEL/UPDATES,267353,nan,nan,nan,nan,nan,129356,79976,285271,349778,2,NaN,NaN,69,8903,nan,nan,nan,nan,nan,nan,17641,6361,24525,20067,2,NaN,NaN,31,7366,nan,nan,nan,nan,nan,nan,14126,34033,125703,181847,0,NaN,NaN,0,792,nan,nan,nan,nan,nan,nan,1258,507,1207,3218,NaN,NaN,NaN,0,445,nan,nan,nan,nan,nan,nan,nan,520221,55670,85660,33282,33818,OTHER,311791,nan,nan,nan,nan,nan,16550,1343,13896,403,16,0,40,ADMINISTRATIVE ACTION,501,CONFIRMED PER RESPONSE TO NOTICE,97,OTHER MISCELLANEOUS,254,A9H: A9I:,8130,714,7416,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,nan,8597,557,8040,2819,365,2454,5307,349,4958,471,451,20,nan,2867,134,2733,1787,92,1695,nan,nan,nan,1080,42,1038,nan,77,69,NaN,8,nan,2764,127,2637,1694,85,1609,NaN,NaN,NaN,1070,42,1028,1. B15A+B16A+B17A EQUAL THE TOTAL WE REPORT IN...,97,6,91,91,6,85,6,NaN,6,0,0,0,nan,nan,nan,nan,nan,93,4,89,57,2,55,30,2,28,NaN,NaN,NaN,nan,657976.0,431215,NaN,NaN,NaN,226761,nan,nan,nan,nan,nan,nan,nan,644113,nan,423189,na

In [464]:
#Understand How States Actually Submitted Their EAVS data
eavs_muni = df[['FIPSCode_x', 'State_Abbr']].groupby(['State_Abbr']).count().reset_index()

census_counties = acs[['fips_code','Official USPS Code']].groupby(['Official USPS Code']).count().reset_index()

get_disconnect = census_counties.merge(eavs_muni, how="left", left_on='Official USPS Code', right_on='State_Abbr')

get_disconnect = get_disconnect.drop(['State_Abbr'], axis=1)
get_disconnect.columns = ['State', 'Correct_Census_Count', 'EAVS_Count']
get_disconnect['county_match'] = get_disconnect['Correct_Census_Count'] == get_disconnect['EAVS_Count']

true_matches = get_disconnect.county_match.sum()

get_disconnect.head()

,State,Correct_Census_Count,EAVS_Count,county_match
0,AK,29,1,False
1,AL,67,67,True
2,AR,75,75,True
3,AZ,15,15,True
4,CA,58,58,True


In [466]:
print('There are', 50-true_matches, 'states without county-level info here.')

trouble_states = get_disconnect[get_disconnect['county_match']==False]['State']
trouble_states

There are 9 states without county-level info here.


0     AK
6     CT
14    IL
19    MA
21    ME
24    MO
30    NH
39    RI
46    VT
48    WI
Name: State, dtype: object

In [456]:
#Intelligble Variable Names
acs['total_population'] = acs['DP05_0033E'].astype(int)
acs['black_population'] = acs['DP05_0038E'].astype(int)
acs['hispanic_population'] = acs['DP05_0072E'].astype(int)
acs['mean_household_income'] = acs['DP03_0063E'].astype(int)
acs['total_households'] = acs['DP02_0001E'].astype(float)
acs['median_household_income'] =  acs['DP03_0062E'].astype(int)
acs['median_age'] = acs['DP05_0018E'].astype(float)

#New Variables
acs['num_black_latino'] = acs['black_population'] + acs['hispanic_population']
acs['percent_minority'] = (acs['black_population'] + acs['hispanic_population']) /  acs['total_population']
acs['num_senior'] =  (acs['DP05_0014E'].astype(float) + acs['DP05_0015E'].astype(float) + acs['DP05_0016E'].astype(float) + acs['DP05_0017E'].astype(float)) 
acs['percent_over_60'] = (acs['DP05_0014E'].astype(float) + acs['DP05_0015E'].astype(float) + acs['DP05_0016E'].astype(float) + acs['DP05_0017E'].astype(float)) / acs['total_population']

keep_census = ['fips_code','percent_minority', 'percent_over_60', 'median_household_income','total_households','median_age', 'num_senior', 'num_black_latino', 'total_population',]

In [419]:
full_df = df.merge(acs[keep_census], how="left", left_on="short_fips", right_on="fips_code")

### Overview: State Trends
* Arizona and Wisconsin do not offer same day registration
* Pennsylvania is not required to report its number of poll workers
* Might sub NH for  Michigan

In [422]:
#Last minute fix
full_df['num_in_person'] = full_df['F1b'] 

In [441]:
# Units and Features of Interest
key_states = ['TX','OH', 'AZ', 'PA', 'WI', 'NC','FL', 'MI']

# #Features we want to keep in the final merge / Drop Excess Columns 
keep_vars = ['State_Full','short_fips', 'State_Abbr', 'Jurisdiction_Name', 'num_in_person','num_inactive','num_black_latino','num_senior', 'num_pw_old','difficulty', 
   'voter_per_location', 'voter_per_machine', 'voter_per_pw','total_resources_per_voter',  'wta_counts', 'wait_time','wait_count',  'eligible_voters', 'num_machines',
        #     'nearby_resources',
             
             'frac_pw_old', 'percent_in_person', 'percent_inactive',  'voter_per_machine', 'voter_per_pw', 'total_resources_per_voter', 'wait_time','wait_count', 'percent_minority', 'percent_over_60', 'num_same_day_locations', 'num_machines', 'num_pollworkers', 'num_pw_old', 'voters_removed', 'num_precincts', 'total_mail_ballots', 'percent_early_vote', 'percent_rejected_ballot',  'median_age', 'median_household_income', 'total_population']

#nearbyresources

# Features for Looping and Riskiness
risk_feats = ['short_fips', 'State_Abbr','eligible_voters','num_black_latino','num_senior', 'num_pw_old','difficulty','percent_in_person', 'percent_inactive', 'voter_per_location',  'voter_per_machine', 'voter_per_pw', 'total_resources_per_voter', 'wait_time']

# DF for Key States and Risk Variables
clean_df = full_df[keep_vars]
key_df = full_df[full_df['State_Abbr'].isin(key_states)][keep_vars]
print(key_df.head())


# oh_df = key_df[key_df['State_Abbr']=='OH'][keep_vars]
# oh_df.to_csv('ohio_raw_data.csv')

   State_Full short_fips State_Abbr Jurisdiction_Name  num_in_person  \
69    ARIZONA      04001         AZ     APACHE COUNTY        13440.0   
70    ARIZONA      04003         AZ    COCHISE COUNTY         8591.0   
71    ARIZONA      04005         AZ   COCONINO COUNTY        15391.0   
72    ARIZONA      04007         AZ       GILA COUNTY         3635.0   
73    ARIZONA      04009         AZ     GRAHAM COUNTY         3469.0   

    num_inactive  num_black_latino  num_senior  num_pw_old  difficulty  \
69        4971.0            3859.0     14758.0        99.0         4.0   
70       16381.0           44041.0     36129.0        77.0         4.0   
71       15500.0           17803.0     24202.0       161.0         4.0   
72        7433.0            9515.0     19272.0       161.0         5.0   
73        3250.0           12587.0      7083.0        25.0         3.0   

    voter_per_location  voter_per_machine  voter_per_pw  \
69         1306.666667        1193.043478    197.410072   
70  

In [442]:
# Output Final Risk Indicators Datarame
import pickle

clean_df.to_csv(data_root + 'all_states_risk_data.csv')
key_df.to_csv(data_root+'key_states_risk_data.csv')
key_df.to_pickle('key_states.pkl')
clean_df.to_pickle('all_df.pkl')

In [ ]:
key_df.head()

In [413]:
# Understand Which States Are Missing Which Values
import seaborn as sns
track_state_nas = key_df[risk_feats].set_index('State_Abbr').isna().sum(level=0)
num_counties = key_df[risk_feats].groupby(['State_Abbr']).count()
percent_missing = track_state_nas / num_counties
#sns.heatmap(percent_missing, annot=True)
percent_missing.to_csv('missing_matrix.csv')
track_state_nas.to_csv('count_nas_by_state.csv')
#track_state_nas.to_csv

In [440]:

track_all_nas = full_df[risk_feats].set_index('State_Abbr').isna().sum(level=0)
num_counties = full_df[risk_feats].groupby(['State_Abbr']).count()
percent_missing = track_all_nas / num_counties
percent_missing.to_csv('toolkit_ready_states.csv')

In [436]:
edf = pd.read_excel(data_root +'EAVS 2016 Final Data for Public Release v.4.xls', dtype=str)
print(len(df))


4681


,FIPSCode,State,JurisdictionName,A1a,A1Comments,A2,A2c_Other,A2_Comments,A3a,A3b,...,A11g,A11h_Other,A11h,A11i_Other,A11i,A11j_Other,A11j,A11k_Other,A11k,A11Comments
3318,3900100000,OH,ADAMS,16945,,Active voters only,,,16945,-888888: Not Applicable,...,36,,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,
3319,3900300000,OH,ALLEN,68054,,Active voters only,,,68054,-888888: Not Applicable,...,49,,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,
3320,3900500000,OH,ASHLAND,35452,,Active voters only,,,35452,-888888: Not Applicable,...,60,,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,
3321,3900700000,OH,ASHTABULA,60624,,Active voters only,,,60624,-888888: Not Applicable,...,294,,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,
3322,3900900000,OH,ATHENS,45418,,Active voters only,,,45418,-888888: Not Applicable,...,227,,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,
3323,3901100000,OH,AUGLAIZE,31666,,Active voters only,,,31666,-888888: Not Applicable,...,11,,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,
3324,3901300000,OH,BELMONT,46971,,Active voters only,,,46971,-888888: Not Applicable,...,145,,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,
3325,3901500000,OH,BROWN,28405,,Active voters only,,,28405,-888888: Not Applicable,...,20,,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,
3326,3901700000,OH,BUTLER,247972,,Active voters only,,,247972,-888888: Not Applicable,...,1048,,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,
3327,3901900000,OH,CARROLL,18002,,Active voters only,,,18002,-888888: Not Applicable,...,26,,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,


In [439]:
pd.set_option('display.max_columns', None)
edf[edf['State']=='OH'].head(30)

,FIPSCode,State,JurisdictionName,A1a,A1Comments,A2,A2c_Other,A2_Comments,A3a,A3b,A3Comments,A4a,A4b,A4b_Other,A4Comments,A5a,A5b,A5c,A5d,A5e,A5f,A5g,A5h_Other,A5h,A5i_Other,A5i,A5j_Other,A5j,A5k_Other,A5k,A5l_Other,A5l,A5Comments,A6a,A6b,A6c,A6d,A6e,A6f,A6g,A6h,A6i,A6j_Other,A6j,A6k_Other,A6k,A6l_Other,A6l,A6m_Other,A6m,A6n_Other,A6n,A6o_Other,A6o,A6Comments,A7a,A7b,A7c,A7d,A7e,A7f,A7g,A7h,A7i,A7j_Other,A7j,A7k_Other,A7k,A7l_Other,A7l,A7m_Other,A7m,A7n_Other,A7n,A7o_Other,A7o,A7Comments,A8a,A8b,A8c,A8d,A8e,A8f,A8g,A8h,A8i,A8j_Other,A8j,A8k_Other,A8k,A8l_Other,A8l,A8m_Other,A8m,A8n_Other,A8n,A8o_Other,A8o,A8Comments,A9a,A9b,A9c,A9d,A9e,A9f,A9g,A9h,A9i,A9j_Other,A9j,A9k_Other,A9k,A9l_Other,A9l,A9m_Other,A9m,A9n_Other,A9n,A9o_Other,A9o,A9Comments,A10a,A10b,A10c,A10d,A10e,A10f_Other,A10f,A10g_Other,A10g,A10h_Other,A10h,A10Comments,A11a,A11b,A11c,A11d,A11e,A11f,A11g,A11h_Other,A11h,A11i_Other,A11i,A11j_Other,A11j,A11k_Other,A11k,A11Comments
3318,3900100000,OH,ADAMS,16945,,Active voters only,,,16945,-888888: Not Applicable,,-888888: Not Applicable,-888888: Not Applicable,,,5852,2470,30,1332,156,1864,-888888: Not Applicable,,0,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,,1275,1130,0,2329,711,0,11,396,0,,0,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,,598,509,0,985,220,0,7,151,0,nan,-888888: Not Applicable,nan,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,,303,275,-888888: Not Applicable,482,264,-888888: Not Applicable,0,8,-888888: Not Applicable,,0,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,,43,35,-888888: Not Applicable,19,53,-888888: Not Applicable,0,6,0,,0,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,,4560,1189,-888888: Not Applicable,397,2974,nan,-999999: Data Not Available,nan,-999999: Data Not Available,nan,-999999: Data Not Available,,2344,30,412,53,1813,0,36,,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,
3319,3900300000,OH,ALLEN,68054,,Active voters only,,,68054,-888888: Not Applicable,,-888888: Not Applicable,-888888: Not Applicable,,,24775,9442,98,5825,25,9385,-888888: Not Applicable,,0,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,,2721,4243,0,12810,3880,54,0,1048,19,,0,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,,1367,1833,0,4241,1278,16,0,703,4,nan,-888888: Not Applicable,nan,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,,633,875,-888888: Not Applicable,2561,1551,8,0,186,11,,0,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,,0,24,-888888: Not Applicable,1,0,0,0,0,0,,0,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,,20012,5168,-888888: Not Applicable,1981,12863,nan,-999999: Data Not Available,nan,-999999: Data Not Available,nan,-999999: Data Not Available,,6260,3886,940,142,1242,1,49,,-999999: Data Not Available,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,
3320,3900500000,OH,ASHLAND,35452,,Active voters only,,,35452,-888888: Not Applicable,,-888888: Not Applicable,-888888: Not Applicable,,,13931,5767,0,4412,211,3541,-888888: Not Applicable,,0,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Applicable,nan,-888888: Not Ap

In [435]:
edf['inactive_voters'] = edf['A3b']
oh_edf= edf[edf['State']=='OH'][['FIPSCode','inactive_voters']]
oh_edf.to_csv('ohio_inactive.csv')

oh_edf.head()

,FIPSCode,inactive_voters
3318,3900100000,-888888: Not Applicable
3319,3900300000,-888888: Not Applicable
3320,3900500000,-888888: Not Applicable
3321,3900700000,-888888: Not Applicable
3322,3900900000,-888888: Not Applicable


In [433]:
edf[edf['State_Abbr']=='OH'].iloc[:][:20]

KeyError: 'State_Abbr'